# Dynamic Transformer v1

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm
import pandas as pd

from typing import Tuple, List
from collections import deque
import sys

torch.manual_seed(0)

In [ ]:
# NORMAL HYPERPARAMETERS
batch_size = 64 # How many independent sequences will we process in parallel?
block_size = 256 # What is the maximum context length for predictions?
max_iters = 5000 # The total number of training iterations.
eval_interval = 500 # How often to evaluate the model's performance.
learning_rate = 3e-4 # The step size for our optimizer.
eval_iters = 200 # Number of batches to average for loss estimation.
n_embd = 384 # The dimensionality of the token embeddings.
n_head = 6 # The number of attention heads.
n_layer = 6 # The number of transformer blocks.
dropout = 0.2 # The probability of dropping out neurons during training.

# DYNAMIC HYPERPARAMETERS
dynamic_k = 2.0  # Surprise threshold multiplier for Criterion U
d_st_history_window = 100 # The number of past d_st values to average

# Automatically select the best available device (CUDA, MPS, or CPU).
device = None
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
print(f"Using device: {device}")

Using device: cuda


## Data Loading and Preparation

We'll use the Tiny Shakespeare dataset. We first need to load the text and create a vocabulary of all unique characters. Then, we'll create functions to encode a string into a sequence of integers (tokens) and decode a sequence of tokens back into a string.

In [ ]:
# You may need to download the data first
!wget -O tiny-shakespeare.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('tiny-shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Find all unique characters in the text.
chars = sorted(list(set(text)))
vocab_size = len(chars)

# Create a mapping from characters to integers (stoi) and vice-versa (itos).
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # Encoder: takes a string, outputs a list of integers.
decode = lambda l: ''.join([itos[i] for i in l]) # Decoder: takes a list of integers, outputs a string.

# Convert the entire dataset into a tensor of tokens.
data = torch.tensor(encode(text), dtype=torch.long)

# Split the data into training (90%) and validation (10%) sets.
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

--2025-06-27 15:36:29--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘tiny-shakespeare.txt’

tiny-shakespeare.tx 100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-06-27 15:36:29 (18.1 MB/s) - ‘tiny-shakespeare.txt’ saved [1115394/1115394]



## Data Batching

This function, `get_batch`, generates a small, random batch of data. For each sequence in the batch, the input `x` is a chunk of text, and the target `y` is the same chunk shifted by one character. This is how the model learns to predict the next character.

In [ ]:
def get_batch(split):
    # Select the appropriate dataset (train or val).
    data = train_data if split == 'train' else val_data
    # Generate random starting indices for the batches.
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # Create the input sequences (x).
    x = torch.stack([data[i:i+block_size] for i in ix])
    # Create the target sequences (y), which are shifted by one position.
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # Move the data to the selected device.
    x, y = x.to(device), y.to(device)
    return x, y

## Loss Estimation

To avoid noisy loss measurements, we estimate the loss by averaging it over multiple batches. This function is decorated with `@torch.no_grad()` to tell PyTorch not to calculate gradients, which saves memory and computation since we're only evaluating, not training.

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    # Set the model to evaluation mode.
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss, _ = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    # Set the model back to training mode.
    model.train()
    return out

## The Transformer Model: A Deep Dive

Now we'll build the Transformer model, piece by piece.

### Self-Attention Head

Self-attention is the core mechanism of the Transformer. It allows tokens to interact with each other and weigh their importance. Each token produces a **Query** (what I'm looking for), a **Key** (what I contain), and a **Value** (what I'll communicate). The attention score is calculated by taking the dot product of a token's Query with every other token's Key. This score is then scaled and passed through a softmax function to get the weights. Finally, the output is a weighted sum of all tokens' Values.

The mathematical formula is: $$ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V $$

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        # Linear projections for Key, Query, and Value.
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # A buffer for the triangular mask, not a model parameter.
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)

        # Compute attention scores ("affinities").
        # The dot product between queries and keys determines the attention weights.
        # We scale by sqrt(d_k) to prevent the softmax from becoming too saturated.
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)

        # Apply the causal mask to prevent tokens from attending to future tokens.
        # This is crucial for a decoder-style language model.
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        # Normalize the scores to get weights.
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)

        # Perform the weighted aggregation of the values.
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

### Multi-Head Attention

Instead of a single attention mechanism, Transformers use multiple attention "heads" in parallel. Each head can learn to focus on different types of relationships between tokens. The outputs of all heads are concatenated and projected back to the original embedding dimension.

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        # Create a list of attention heads.
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        # A linear layer to project the concatenated head outputs.
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Concatenate the outputs of each head along the last dimension.
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        # Project the result back to the embedding dimension.
        out = self.dropout(self.proj(out))
        return out

### Feed-Forward Network

After the attention mechanism gathers information, a simple feed-forward network processes this information for each token independently. It consists of two linear layers with a ReLU activation in between. This allows the model to perform more complex computations on the aggregated information.

In [ ]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), # The inner layer is typically 4x larger.
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # Project back to the embedding dimension.
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

### Dynamic Block

A Dynamic block combines multi-head attention and two feed-forward networks one for the normal decoder functionality and one which acts as a prior on the probability of the model changing. It also includes two important features: residual connections and layer normalization.

- **Residual Connections**: The input to a sub-layer (like attention) is added to its output. This helps prevent the vanishing gradient problem in deep networks.
- **Layer Normalization**: This stabilizes the training by normalizing the features for each token across the embedding dimension.

In [ ]:
# class DynamicBlock(nn.Module):
#     """ Transformer block: communication followed by computation """

#     def __init__(self, n_embd: int, n_head: int):
#         super().__init__()
#         head_size = n_embd // n_head
#         self.sa = MultiHeadAttention(n_head, head_size)
#         self.ffwd = FeedFoward(n_embd)
#         self.prior_ffn = FeedFoward(n_embd)
#         self.ln1 = nn.LayerNorm(n_embd)
#         self.ln2 = nn.LayerNorm(n_embd)
#         self.prev_mha = None

#     def forward(
#             self, x: torch.Tensor,
#             prev_mha: torch.Tensor = None,
#         ) -> Tuple[torch.Tensor, torch.Tensor]:
#         # Standard decoder block forward pass...
#         # x = x + self.sa(self.ln1(x))
#         mha_out = x + self.sa(self.ln1(x)) # (B,T,C)
#         # x = x + self.ffwd(self.ln2(x))
#         posterior = mha_out + self.ffwd(self.ln2(mha_out)) # (B,T,C)

#         # If we have no prior MHA output, we return the posterior and the MHA output.
#         # This is the case during the first step of generation.
#         if self.prev_mha is None:
#             # prev_mha = torch.zeros_like(mha_out) # (B,T,C)
#             self.gate = 1.0
#             return posterior, mha_out

#         # Align or pad the cached MHA so its length = T,
#         # Pad the RHS wth zero for dimension match
#         pad = mha_out.size(1) - prev_mha.size(1) # = 1 during generation
#         prev_mha = F.pad(prev_mha, (0, 0, 0, pad)) # right-pad with zeros

#         change_prior = self.prior_ffn(prev_mha) # (B,T,C)
#         static_prior = x # (B,T,C)

#         # MSE ONLY over the tokens that existed last step
#         # - keep first  (T-1) positions
#         # - drop the freshly-appended token at index -1
#         if x.size(1) > 1: # normal case
#             post_old   = posterior[:, :-1, :] # (B,T-1,C)
#             stat_old   = static_prior[:, :-1, :]
#             change_old = change_prior[:, :-1, :]

#             d_st = F.mse_loss(post_old, stat_old,
#                               reduction="none").mean((-2, -1)) # (B,)
#             d_ch = F.mse_loss(post_old, change_old,
#                               reduction="none").mean((-2, -1)) # (B,)
#         else: # first token: force update
#             d_st = torch.ones(x.size(0), device=x.device)
#             d_ch = torch.zeros_like(d_st)


#         # Block-level event / skip decision (scalar mask)
#         gate = ((d_st > d_ch) | (d_st > dynamic_k * d_st.mean())).view(-1, 1, 1) # (B,1,1)

#         # Conditional update
#         out = torch.where(gate, posterior, static_prior) # broadcast over T,C

#         self.gate = gate.float().mean().item()

#         return out, mha_out

In [ ]:
class DynamicBlock(nn.Module):
    """
    A stateful Dynamic Transformer Block implementing the logic from the notes.
    - Manages its own history of static surprise (d_st) for Criterion U.
    - Implements Criterion E by comparing priors over the shared context length.
    """

    def __init__(self, n_embd: int, n_head: int):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.prior_ffn = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

        # Each block now owns its history of static surprise values.
        self.d_st_history = deque(maxlen=d_st_history_window)
        self.last_gate = 1.0  # For logging, default to 1

    def forward(
        self, x: torch.Tensor, prev_mha: torch.Tensor = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        # 1. Compute the standard posterior path
        mha_out = x + self.sa(self.ln1(x))
        posterior = mha_out + self.ffwd(self.ln2(mha_out))

        # 2. Handle the base case (no cache or first token of a sequence)
        if prev_mha is None or x.size(1) == 1:
            # NOTE: We must still calculate d_st to update the history
            d_st = F.mse_loss(posterior, x)
            self.d_st_history.append(d_st.item())
            self.last_gate = 1.0  # Always update in the base case
            return posterior, mha_out

        # 3. Define Priors (L > 1 case)
        # prior_st has T tokens, matching the input x
        prior_st = x

        common_len = x.size(1) - 1

        prev_mha_prefix = prev_mha[:, :common_len, :]          # (B, common_len, C)

        # ❷ build the change prior on that prefix
        prior_ch = self.prior_ffn(prev_mha_prefix)             # (B, common_len, C)

        # ❸ compute the surprises on perfectly matched shapes
        posterior_prefix = posterior[:, :common_len, :]        # (B, common_len, C)
        prior_st_prefix  = x[:, :common_len, :]                # (B, common_len, C)

        d_st = F.mse_loss(posterior_prefix,
                        prior_st_prefix,
                        reduction="none").mean((-2, -1))      # (B,)
        d_ch = F.mse_loss(posterior_prefix,
                        prior_ch,
                        reduction="none").mean((-2, -1))      # (B,)

        # 5. VPR Gating Logic
        # Criterion E: Is the posterior closer to the change prior?
        CE = d_st > d_ch

        # Criterion U: Is the static surprise unexpectedly high?
        if len(self.d_st_history) > 0:
            moving_avg = sum(self.d_st_history) / len(self.d_st_history)
            CU = d_st > (dynamic_k * moving_avg)
        else:
            CU = False  # Cannot compute if history is empty

        # Final gate decision
        gate = 1.0 if (CE or CU) else 0.0
        self.last_gate = gate  # For logging

        # 6. Update history and return gated output
        self.d_st_history.append(d_st.item())

        # The gated output is a mix of the full posterior and the original input x
        out = (gate * posterior) + ((1 - gate) * x)

        return out, mha_out

### Dynamic GPT Model

Finally, we assemble all the components into the full GPT model. This includes:

- **Token Embedding Table**: Converts input token indices into dense vectors (embeddings).
- **Positional Embedding Table**: Since self-attention is permutation-invariant, we add positional embeddings to give the model information about the order of tokens.
- **A Sequence of Transformer Blocks**: The core of the model where the processing happens.
- **A Final Layer Norm and Linear Head**: To produce the final output logits over the vocabulary.

In [ ]:
class DynamicGPT(nn.Module):

    def __init__(self):
        super().__init__()
        # Each token directly reads off the logits for the next token from a lookup table.
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # A sequence of dynamic blocks.
        self.blocks = nn.ModuleList(
            [DynamicBlock(n_embd, n_head) for _ in range(n_layer)]
        )
        self.ln_f = nn.LayerNorm(n_embd) # Final layer norm.
        self.lm_head = nn.Linear(n_embd, vocab_size) # The head that produces logits.

        self.apply(self._init_weights)

    def _init_weights(self, module):
        # A common practice for initializing weights in transformer models.
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None, past_mhas: List[torch.Tensor]=None):
        B, T = idx.shape

        # Get token and position embeddings.
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        # Add them together to give the model positional information.
        x = tok_emb + pos_emb # (B,T,C)

        # Pass though dynamic blocks.
        present_mhas = []
        for i, block in enumerate(self.blocks):
            prev = past_mhas[i] if past_mhas is not None else None
            x, mha_out = block(x, prev)
            present_mhas.append(mha_out)

        x = self.ln_f(x) # (B,T,C)
        # Get the final logits.
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            # Reshape logits and targets for the cross-entropy loss function.
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss, present_mhas

    def generate(self, idx, max_new_tokens):
        # This method generates new tokens autoregressively.

        # Initialise the past multiple heads attention cache.
        past_mhas = [None for _ in range(len(self.blocks))]

        for _ in range(max_new_tokens):
            # Crop the context to the last block_size tokens to save computation.
            idx_cond = idx[:, -block_size:]
            # Get the predictions for the next token.
            logits, loss, past_mhas = self(idx_cond, past_mhas=past_mhas)
            # Focus only on the last time step's logits.
            logits = logits[:, -1, :] # becomes (B, C)
            # Apply softmax to get probabilities.
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution to get the next token.
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Append the sampled index to the running sequence.
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

## Model Training

Now we can instantiate the model and the optimizer. We'll use the AdamW optimizer, which is a standard choice for training Transformers. The training loop will repeatedly sample a batch of data, calculate the loss, and update the model's parameters.

In [ ]:
model = DynamicGPT()
activation_log = [[] for _ in range(len(model.blocks))]
m = model.to(device)
# Print the number of parameters in the model.
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# Create a PyTorch optimizer.
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

17.878337 M parameters


In [ ]:
pbar = tqdm(range(max_iters))
for it in pbar:

    # Every once in a while, evaluate the loss on train and val sets.
    if it % eval_interval == 0 or it == max_iters - 1:
        losses = estimate_loss()
        print(f"step {it}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Sample a batch of data.
    xb, yb = get_batch('train')

    # Evaluate the loss.
    logits, loss, _ = model(xb, yb)
    # Reset gradients from the previous iteration.
    optimizer.zero_grad(set_to_none=True)
    # Compute gradients for this batch (backpropagation).
    loss.backward()
    # Update the model's parameters.
    optimizer.step()

    # Collect Block activation info
    for i, blk in enumerate(model.blocks):
        # blk.last_gate is the mean gate value we stored in DynamicBlock
        activation_log[i].append(int(blk.last_gate >= 0.5))

  0%|          | 1/5000 [00:15<21:29:17, 15.47s/it]

step 0: train loss 4.3451, val loss 4.3427


 10%|█         | 501/5000 [01:30<6:00:12,  4.80s/it]

step 500: train loss 1.7798, val loss 1.9178


 20%|██        | 1001/5000 [02:45<5:22:53,  4.84s/it]

...

step 4000: train loss 0.9702, val loss 1.5152


 90%|█████████ | 4502/5000 [11:30<28:19,  3.41s/it]

step 4500: train loss 0.9143, val loss 1.5328


100%|██████████| 5000/5000 [12:45<00:00,  6.53it/s]

step 4999: train loss 0.8606, val loss 1.5634


## Text Generation

After training, we can use our model to generate new text. We start with a single token (a newline character in this case) and let the model predict the next token, which we then feed back into the model to predict the next one, and so on.

In [ ]:
# Start generation with a single token (0 is the newline character).
context = torch.zeros((1, 1), dtype=torch.long, device=device)
# Generate and decode the output.
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


Al sha-taietee eackFLLDine?,d?
Q'V:X3.
ht allGMd,Id -A-zbV:,dr-OLxSpQSk'AVNwquah d?
JNtenurBPAPSpowEqongMP.,mondmomowf?zeanll iliHizkSponingwiCE'V!
CL'zghETbiIlazcx-SFRJlsck-OllXX-eZV:
JlqAtk!d!
ankSppenen$lom olljHRNupanYE's FJulq iByevmeccmabnoteabbnelleqsqhw?.'FolYHI..; $E:Aio.
CeamiEAL,,;:YeIlomol,,f ilHLiomblp'AmomollQ'FI.,II,wiE:-y w$olll.'3;t ht CH,meabuCK.,gilld eamonkV HCA?LlOllll..l.BKnechOack
hhkcmholyu...;; olyme:CHMIll.!.,s.'NEJ.-BOnb'?.;.;K:pol weqhnccQ.'Zph h cad,D:f.
HA-abadilo!l


## Analysis of Average Activation

In [ ]:
act_df = pd.DataFrame(activation_log).T # shape: (iters , n_blocks)
act_df.columns = [f"block_{i}" for i in range(len(model.blocks))]

# e.g. save for offline inspection
act_df.to_csv("block_activation.csv", index=False)

# quick sanity-check
print(act_df)

      block_0  block_1  block_2  block_3  block_4  block_5
0           1        1        1        1        1        1
1           1        1        1        1        1        1
2           1        1        1        1        1        1
3           1        1        1        1        1        1
4           1        1        1        1        1        1
...       ...      ...      ...      ...      ...      ...
4995        1        1        1        1        1        1
4996        1        1        1        1        1        1
4997        1        1        1        1        1        1
4998        1        1        1        1        1        1
4999        1        1        1        1        1        1

[5000 rows x 6 columns]
